##### 01. Landing to LH_Bronze


In [1]:
from pyspark.sql.functions import *

StatementMeta(, 0f02669f-ca32-4a00-9a43-c24bdd8f2578, 3, Finished, Available, Finished)

In [2]:
toay_file = 'UTIL_AREA_2025-01-28.parquet'
processed_date = '2025-01-28'
# workspace = 'a3e4d0e6-6b32-498b-addc-1629a4b29a47'
workspace = 'Fabric Trial'

StatementMeta(, 0f02669f-ca32-4a00-9a43-c24bdd8f2578, 4, Finished, Available, Finished)

In [3]:
#check in your accountName, ontainerName, relativePath to files and your adls endpoint.
act_Name = 'stakpepuksdataengdev01'
cnt_Name = 'bronze'
rlt_Path = 'kphqgen04_Bail_01/MetaDataSource'

adls_Path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (cnt_Name, act_Name, rlt_Path)

source_path = f"{adls_Path}/{toay_file}"

#checkes
print(adls_Path)
print(source_path)



StatementMeta(, 0f02669f-ca32-4a00-9a43-c24bdd8f2578, 5, Finished, Available, Finished)

abfss://bronze@stakpepuksdataengdev01.dfs.core.windows.net/kphqgen04_Bail_01/MetaDataSource
abfss://bronze@stakpepuksdataengdev01.dfs.core.windows.net/kphqgen04_Bail_01/MetaDataSource/UTIL_AREA_2025-01-28.parquet


##### Reading Landed file

In [4]:
df = spark.read.parquet(source_path)
display(df)
df.printSchema()


StatementMeta(, 0f02669f-ca32-4a00-9a43-c24bdd8f2578, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 573b5278-1813-4d2d-92d5-e3abc7221921)

root
 |-- id: decimal(19,0) (nullable = true)
 |-- police_force: string (nullable = true)
 |-- area: string (nullable = true)



In [5]:
if df.count() > 1:
    print("The file has data.")

    df_pdate = df.withColumn("Processing_Date", lit(processed_date))
   # display(df_pdate)
    df_pdate.createOrReplaceTempView('temp_table')

else:
    print("The file is blank")

StatementMeta(, 0f02669f-ca32-4a00-9a43-c24bdd8f2578, 7, Finished, Available, Finished)

The file has data.


In [6]:
# -- %%sql
# -- select * from temp_table

StatementMeta(, 0f02669f-ca32-4a00-9a43-c24bdd8f2578, 8, Finished, Available, Finished)

In [7]:
#Fabric path
# Lh_bnx_path = f"abfss://a3e4d0e6-6b32-498b-addc-1629a4b29a47@onelake.dfs.fabric.microsoft.com/85f3437b-a6b9-42ba-aff4-eebf805473fd/Tables/UTIL_AREA"
Lh_bnx_path = f"abfss://{workspace}@onelake.dfs.fabric.microsoft.com/lh_A4E_02/Tables/UTIL_AREA"

#Create table if not exist 
try:
    spark.read.format("Delta").load(Lh_bnx_path).createOrReplaceTempView('UTIL_AREA')

except:
    create_table = f"""CREATE TABLE IF NOT EXISTS UTIL_AREA
    (
    ID INT,
    Police_Force STRING,
    Area STRING,
    Processing_Date DATE
    )""" 
spark.sql(create_table)

# spark.read.format("Delta").load(Lh_bnx_path).createOrReplaceTempView('UTIL_AREA')


StatementMeta(, 0f02669f-ca32-4a00-9a43-c24bdd8f2578, 9, Finished, Available, Finished)

DataFrame[]

##### Update Bronze Table

In [8]:
update_sql = f"""
    MERGE INTO util_area AS t
    USING temp_table AS s 
    ON t.ID = s.id and t.Police_Force = s.Police_Force

    WHEN MATCHED THEN
        UPDATE SET
            t.ID = s.id
            ,t.Police_Force = s.Police_Force
            ,t.Area = s.Area
            ,t.Processing_Date = '{processed_date}'
    
    WHEN NOT MATCHED THEN
    INSERT(ID, Police_Force, Area, Processing_Date)
    VALUES(
            s.id
            ,s.Police_Force
            ,s.Area
            ,'{processed_date}'
          )
        """

spark.sql(update_sql).show()

StatementMeta(, 0f02669f-ca32-4a00-9a43-c24bdd8f2578, 10, Finished, Available, Finished)

+-----------------+----------------+----------------+-----------------+
|num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
|                6|               6|               0|                0|
+-----------------+----------------+----------------+-----------------+



In [9]:
%%sql
SELECT * FROM util_area

StatementMeta(, 0f02669f-ca32-4a00-9a43-c24bdd8f2578, 11, Finished, Available, Finished)

<Spark SQL result set with 6 rows and 4 fields>